# Train Test Split und Cross Validation Lösung

In [29]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [30]:
pd.set_option('display.max_columns', 21)

### Daten einlesen

In [31]:
spam_df = pd.read_csv('https://vincentarelbundock.github.io/Rdatasets/csv/openintro/email.csv', index_col=0)

### Variable Explanation
| Variable | Explanation |
|--- | --- |
| spam | Indicator for whether the email was spam. |
| to_multiple | Indicator for whether the email was addressed to more than one recipient. |
| from | Whether the message was listed as from anyone (this is usually set by default for regular outgoing email). |
| cc | Number of people cc'ed. |
| sent_email | Indicator for whether the sender had been sent an email in the last 30 days. |
| time | Time at which email was sent. |
| image | The number of images attached. |
| attach | The number of attached files. |
| dollar | The number of times a dollar sign or the word “dollar” appeared in the email. |
| winner | Indicates whether “winner” appeared in the email. |
| inherit | The number of times “inherit” (or an extension, such as “inheritance”) appeared in the email. |
| viagra | The number of times “viagra” appeared in the email. |
| password | The number of times “password” appeared in the email. |
| num_char | The number of characters in the email, in thousands. |
| line_breaks | The number of line breaks in the email (does not count text wrapping). |
| format | Indicates whether the email was written using HTML (e.g. may have included bolding or active links). |
| re_subj | Whether the subject started with “Re:”, “RE:”, “re:”, or “rE:” |
| exclaim_subj | Whether there was an exclamation point in the subject. |
| urgent_subj | Whether the word “urgent” was in the email subject. |
| exclaim_mess | The number of exclamation points in the email message. |
| number | Factor variable saying whether there was no number, a small number (under 1 million), or a big number. |


In [32]:
random_seed = 5

### Daten vorbereiten

In [33]:
spam_df.drop(columns='time', inplace=True)

#### Binäre kategoriale Variablen in Dummy Variablen umwandeln (0-1-Encodierung) 

In [34]:
spam_df = pd.get_dummies(spam_df, columns=['winner', 'number'], drop_first=True)


#### Downsampling


Downsampling, die gezielte Reduzierung des Datasets, ist hier aus zweierlei Hinsicht nützlich. Zum einen erleichtert es das Vorgehen im Folgenden, da so ein ausgeglichenes Dataset hinsichtlich der Zielvariable erstellt werden kann. Zum anderen reduziert ein kleineres Dataset natürlich auch die Zeit die das wiederholte Training erfordert.

In [35]:
num_spam_labels = 250  # spam_df.spam.sum()

spam_df_downsampled = spam_df.groupby('spam').sample(n=num_spam_labels, random_state=random_seed).reset_index(drop=True)

In [36]:
spam_df_downsampled

,spam,to_multiple,from,cc,sent_email,image,attach,dollar,inherit,viagra,password,num_char,line_breaks,format,re_subj,exclaim_subj,urgent_subj,exclaim_mess,winner_yes,number_none,number_small
0,0,1,1,0,0,0,0,0,0,0,0,11.639,240,1,0,0,0,5,0,0,1
1,0,1,1,0,0,0,0,0,0,0,0,39.805,499,0,1,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,4.614,51,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,0,1.118,13,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,0,13.008,194,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,0,1,0,0,0,0,0,0,0,0,2.311,59,1,0,0,0,1,0,0,1
496,1,0,1,0,0,0,0,0,0,0,0,13.521,247,1,0,0,0,16,0,0,1
497,1,0,1,0,0,0,0,2,0,0,0,1.377,43,0,0,0,0,0,0,0,1
498,1,0,1,0,0,0,0,0,0,0,0,0.775,15,1,0,0,0,1,0,0,1


### Training und Validierung

#### 1. Erstellen Sie als erstes X und y Variablen. Transformieren Sie dafür die Zielvariable `'spam'` aus `spam_df_downsampled` zu einem Numpy Array mit der `to_numpy()` Methode. Tun Sie das gleiche für die restlichen Variablen und speichern Sie sie in X ab.

### 

In [37]:
y = spam_df_downsampled['spam'].to_numpy()
X = spam_df_downsampled.drop(columns='spam').to_numpy()

#### 2.1 Als erstes nehmen wir die `train_test_split` Funktion unter die Lupe. Verwenden Sie die Funktion mit X, y und einer Testsetgröße von 0.2, sprich 20%. Speichern Sie die zurückgegebenen Objekte in den jeweils passenden Variablen `X_train, X_test, y_train, y_test` ab. Geben Sie danach die Zeilen und Spalten Anzahl (`shape`) der jeweiligen Variablen aus und überprüfen Sie sie auf Plausibilität.

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(400, 20) (100, 20)
(400,) (100,)


#### 2.2 Nun können Sie `train_test_splitt` anwenden. Verwenden Sie im folgenden die Funktion `fit_predict` um Modelle zu trainieren, Vorhersagen für das Testset zu erstellen und den Wert der Accuracy Metrik auszugeben. Nutzen Sie innerhalb eines for-Loops erst `train_test_split` zum erstellen der Trainings- und Testvariablen, wenden Sie dann `fit_predict` an um die Accuracy für die Modelle Random Forest und Decission Tree zu berechnen und speichern Sie schließlich die Ergebnisse zur späteren Analyse ab, bspw. in einem `Dictionary` oder entsprechenden `List`s.
#### Dabei sollten es 10 Wiederholungen sein und idealerweise vergeben Sie bei jeder Wiederholung einen vorgegebenen, aber sich zwischen den Wiederholungen unterscheidenden, `random_state` für `train_test_split` und `fit_predict`. Dieser Random State, eine ganze Zahl >=0, ermöglicht eine Replizierbarkeit Ihrer Ergebnisse. Eine einfache Möglichkeit dafür ist zum Beispiel die Ausgabe der `range()` Funktion als `random_state` zu verwenden.

In [39]:
def init_models(random_state=None):
    '''Initializes models with set parameters and returns them in a dictionary'''
    tree_model = DecisionTreeClassifier(max_depth=5, max_features=6, random_state=random_state)
    rf_model = RandomForestClassifier(100, max_depth=15, max_features=6, random_state=random_state)

    return {'tree': tree_model, 'rf': rf_model}

def fit_predict(X_train, X_test, y_train, y_test, model_type: str, random_state=None):
    '''
    Initilazes and fits model on given train data and then predicts on test data
    
    returns: test predictions accuracy
    '''
    models = init_models(random_state)
    model = models[model_type]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)

    return test_accuracy



In [40]:
results_tts = {'rf' : [], 'tree': []}
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

    rf_accuracy = fit_predict(X_train, X_test, y_train, y_test, model_type='rf', random_state=i)
    tree_accuracy = fit_predict(X_train, X_test, y_train, y_test, model_type='tree', random_state=i)

    results_tts.get('rf').append(rf_accuracy)
    results_tts.get('tree').append(tree_accuracy) 



#### 2.3 Als nächstes können Sie jetzt Ihre Ergebnisse untersuchen. Am einfachsten ist es einen Pandas Dataframe zu erstellen in dem Sie die Ergebnisse des jeweiligen Modells in einer Spalte speichern. Berechnen Sie anschließend die paarweise Differenz zwischen Random Forest und Decision Tree Ergebnissen für jeden Durchgang um die größe der Abweichungen zu erkennen und geben Sie das Ergebnis aus.

In [41]:
results_train_test_split = pd.DataFrame(results_tts)

results_train_test_split['diff'] = results_train_test_split.rf - results_train_test_split.tree
results_train_test_split

,rf,tree,diff
0,0.84,0.77,0.07
1,0.87,0.84,0.03
2,0.87,0.82,0.05
3,0.88,0.78,0.10
4,0.82,0.79,0.03
5,0.88,0.86,0.02
6,0.82,0.71,0.11
7,0.81,0.78,0.03
8,0.80,0.64,0.16
9,0.87,0.76,0.11


#### 2.4 Zuletzt können Sie nun die Standardabweichung (`str`) der jeweiligen Spalten berechnen. Wir werden sie im Folgenden mit den Werten der anderen Methode, Cross Validation, vergleichen.

In [42]:
std_train_test_split =  results_train_test_split.std().round(3)
print(f'Standard diviation of difference for train test split: \n{std_train_test_split}')

Standard diviation of difference for train test split: 
rf      0.031
tree    0.064
diff    0.047
dtype: float64


Wenig überraschend variieren die Werte des Entscheidungsbaummodells stärker als die des Random Forests, da die Zusammenfassung mehrere Bäume im RF die Varianz, bzw. die Standardabweichung, der Ergebnisse senkt. 

#### 3.1 Im folgenden werden wir die Validierungsmethode Cross Validation (CV) verwenden. Sklearn bietet verschiedene Formen der CV an und es gibt ebenso unterschiedliche Methoden Sie zu implementieren. Wir verwenden als erstes die Klasse `KFold`, die hilft ein Dataset in mehrere gleichgroße Abschnitte (Folds) zu unterteilen. Probieren Sie es als erstes aus indem Sie ein KFold Objekt mit 5 Folds und gemischt (shuffle) initialisieren und in einer Variable speichern. Anschließend können Sie die `split()` Methode verwenden um Train- und Testindizes für X zu erlangen und mit der `next()` Funktion, wie in einem for-Loop, das nächste Element auszugeben, was in diesem Fall die beiden Variablen der Indizes sind.
#### Benennen Sie die Variablen entsprechend mit `train_id, test_id` und geben Sie die `shape` der beiden aus. Außerdem können Sie einen Blick auf die ersten 10 Elemente werfen um einen besseren Eindruck zu bekommen. Falls Sie bei einem der Schritte Hilfe benötigen bietet es sich immer an die `help()` Funktion zu bemühen. Die Dokumentation von KFold enthält etwa auch ein Beispiel wie es angewendet wird.

In [43]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=None)

In [44]:
train_id, test_id = next(k_fold.split(X))

print(train_id.shape, test_id.shape)
print(train_id[:10])
print(test_id[:10])

(400,) (100,)
[ 1  2  3  6  7  8  9 10 11 12]
[ 0  4  5 14 15 17 19 36 38 41]


#### 3.2 Lassen Sie uns jetzt CV anwenden. Initialisieren Sie `KFold` und nutzen Sie anschließend einen Loop um nach und nach die Indizes der jeweiligen Fold auszugeben. Diese können Sie dann verwenden um X und y mit ihnen zu indizieren und so ` X_train, X_test` bzw.  `y_train, y_test` zu erstellen. Darauf folgend können Sie wie zuvor die `fit_predict` Funktion nutzen um mit dem jeweiligen Modell einen Accuracy Score auszugeben. Zuletzt muss noch der Mittelwert (`np.mean()`) der Accuracy Werte über alle Folds pro Modell berechnet werden.

In [45]:
model_types = ['rf', 'tree']

In [46]:
def custom_cross_validate(X, y, model_types, random_state=None):
    k_fold = KFold(n_splits=5, shuffle=True, random_state=random_state)
    fold_results = {model_type: [] for model_type in model_types}

    for train_id, test_id in k_fold.split(X):
        X_train, X_test = X[train_id], X[test_id]
        y_train, y_test = y[train_id], y[test_id]

        for model_type in model_types:
            accuracy = fit_predict(X_train, X_test, y_train, y_test, model_type, random_state=random_state)
            fold_results.get(model_type).append(accuracy)
    
    single_cv_results = {model_type: np.mean(metric_values).round(3) for model_type, metric_values in fold_results.items()}

    return single_cv_results

In [47]:
single_cv_results = custom_cross_validate(X, y, model_types, random_state=1)
print(single_cv_results)

{'rf': 0.838, 'tree': 0.828}


#### 3.3 Da wir CV mit Train-Test-Split vergleichen möchten, muss auch CV 10 mal durchgeführt. Erstellen Sie also wieder einen Loop mit 10 Wiederholungen in dem die Accuracy pro Modell durch CV ermittelt und gespeichert wird. Nutzen Sie dafür gerne auch den in der vorherigen Teilaufgabe geschriebenen Code wieder. Unabhängig davon auf welche Art und Weise Sie es implementieren, ist das Ziel am Ende 10 Werte pro Modell zu haben, die jeweils den Mittelwert aller Folds (5) eines kompletten CV Durchgangs abbilden. Verwenden Sie idealerweise auch hier wieder `random_state`s um Ihre Ergebnisse replizierbar zu machen.

In [48]:
results_cv = {'rf' : [], 'tree': []}
for i in range(10):
    single_cv_results = custom_cross_validate(X, y, model_types, random_state=i)
    for model_type in model_types:
        results_cv.get(model_type).append(single_cv_results[model_type])

In [49]:
# alternative without use of function or dictionary comprehension:
# results_cv = {'rf' : [], 'tree': []}
# for i in range(10):
#     k_fold = KFold(n_splits=5, shuffle=True, random_state=i)
#     single_cv_results = {'rf' : [], 'tree': []}

#     for train_id, test_id in k_fold.split(X):
#         X_train, X_test = X[train_id], X[test_id]
#         y_train, y_test = y[train_id], y[test_id]

#         for model_type in model_types:
#             accuracy = fit_predict(X_train, X_test, y_train, y_test, model_type, random_state=i)
#             single_cv_results.get(model_type).append(accuracy)

#     for model_type in model_types:
#         results_cv.get(model_type).append(np.mean(single_cv_results[model_type]))


#### 3.4 Nachdem Sie alle Accuracy Werte gesammelt haben, können Sie nun wieder einen Pandas Dataframe erstellen und die Differenz beider Spalten berechnen. 

In [50]:
results_cv = pd.DataFrame(results_cv)
results_cv['diff'] = results_cv.rf - results_cv.tree
results_cv

,rf,tree,diff
0,0.844,0.796,0.048
1,0.838,0.828,0.010
2,0.826,0.814,0.012
3,0.848,0.768,0.080
4,0.840,0.784,0.056
5,0.840,0.826,0.014
6,0.838,0.770,0.068
7,0.840,0.788,0.052
8,0.840,0.756,0.084
9,0.840,0.798,0.042


#### 3.5 Berechnen Sie abschließend die Standardabweichungen aller Spalten und vergleichen Sie sie mit denen der Train-Test-Split Ergebnisse zuvor. Was fällt Ihnen auf?

In [51]:
std_cv =  results_cv.std().round(3)
print(f'Standard diviation of difference for: \nCross Validation \n{std_cv} \n \n Train-Test-Split \n{std_train_test_split}')

Standard diviation of difference for: 
Cross Validation 
rf      0.006
tree    0.025
diff    0.027
dtype: float64 
 
 Train-Test-Split 
rf      0.031
tree    0.064
diff    0.047
dtype: float64


Man sieht schnell auf den ersten Blick, dass die Standardabweichungen der Cross Validation durchgehend niedriger sind. Das bedeutet, es gibt weniger Schwankungen und die Ergebnisse sind verlässlicher als mit nur einem Train-Test-Split. 

#### 4. Cross Validation Funktion
#### Scikit-Learn bietet auch eine Funktion die die CV komplett übernehmen kann: `cross_validate()`. Diese ist zwar nicht so flexibel wie die Nutzung von `KFold`, benötigt dafür aber deutlich weniger Zeilen Code.
#### Der Einfachheit halber, berechnen Sie dieses mal nur einmal die Accuracy der jeweiligen Modelle durch CV und speichern Sie. Alles was Sie dafür tun müssen, ist die `cross_validate()` Funktion mit dem gewünschten initilaisierten Modell, X, y, und der gewünschten `scoring` Funktion (hier also `'accuracy'`) abzurufen und das Ergebnis in einer Variable zu speichern. Zum initialisieren der Modelle können Sie die anfangs definierte Funktion `init_models()` verwenden. Das Ergebnis der CV wird als Dictionary ausgegeben aus dem Sie mit dem Key 'test_score' einen Numpy Array mit den Scores pro Fold bekommen, von dem Sie dann nur noch den Mittelwert berechnen müssen, bevor Sie die CV-Accuracy eines Modells speichern.

In [52]:
cv_models = init_models(1)

results_cv_fct = {}
for model_type in model_types:
    cv_obj = cross_validate(estimator=cv_models[model_type] , X=X, y=y, scoring='accuracy')
    mean_accuracy = np.mean(cv_obj['test_score'])
    results_cv_fct[model_type] = mean_accuracy.round(3)

---

### 5. Hyperparameter CV Grid Search

#### Verwenden Sie die sklearn Klasse `GridSearchCV` um eine Parametersuche über ein Netz an Parameterkombinationen durchzuführen, deren Ergebnisse durch Cross Validation validiert werden.

#### Initialisieren Sie dafür als erstes den gewünschten Classifier (bspw. `RandomForestClassifier`) und danach das `GridSearchCV` Objekt mit dem Classifier Objekt sowie `param_grid` und `scoring` als Argumente . Danach kann die Grid Search mit den Trainingsdaten gefittet werden und das Ergebnis als Attribute ausgegeben werden. Welche Parameterkombinationen führen bei Ihnen zur besten Accuracy?  

In [ ]:
random_state = 7
metric = "accuracy"

In [ ]:
params = {'max_depth': [1, 2], 
          'max_features': [3, 4]
          } 

In [ ]:
tree_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()


In [ ]:
np.random.seed(random_seed)
grid_search = GridSearchCV(rf_model, param_grid=params, scoring=metric)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [1, 2], 'max_features': [3, 4]},
             scoring='accuracy')

In [ ]:
num_top_values = 5
pd.DataFrame({"parameters": grid_search.cv_results_['params'],
              metric: grid_search.cv_results_['mean_test_score'].round(4)}).sort_values(metric, ascending=False).iloc[:num_top_values, :]

,parameters,accuracy
2,"{'max_depth': 2, 'max_features': 3}",0.7975
3,"{'max_depth': 2, 'max_features': 4}",0.7900
1,"{'max_depth': 1, 'max_features': 4}",0.7675
0,"{'max_depth': 1, 'max_features': 3}",0.7525


In [ ]:
grid_search.best_params_

{'max_depth': 2, 'max_features': 3}

In [ ]:
grid_search.best_score_

0.7975000000000001

In [ ]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.094125,0.009753,0.007830,0.000115,1,3,"{'max_depth': 1, 'max_features': 3}",0.6875,0.775,0.800,0.7000,0.8000,0.7525,0.048990,4
1,0.080908,0.000434,0.007315,0.000086,1,4,"{'max_depth': 1, 'max_features': 4}",0.6750,0.800,0.825,0.7000,0.8375,0.7675,0.066895,3
2,0.084897,0.002686,0.007492,0.000199,2,3,"{'max_depth': 2, 'max_features': 3}",0.7750,0.800,0.825,0.7750,0.8125,0.7975,0.020000,1
3,0.082871,0.001241,0.007365,0.000121,2,4,"{'max_depth': 2, 'max_features': 4}",0.7125,0.800,0.825,0.7875,0.8250,0.7900,0.041382,2
